In [ ]:
%load_ext Cython

In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Fake data

In [ ]:
def circular_mask(n, r):
    mask = np.zeros((n, n),dtype=bool)
    for i in range(n):
        for j in range(n):
            if (i - n / 2)**2 + (j - n / 2)**2 < r**2:
                mask[i][j]=True
    return mask

# New code

In [ ]:
%%cython --annotate

import numpy as np
cimport numpy as np

cimport cython

from libc.stdlib cimport malloc, calloc, free

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef get_start(np.ndarray[np.uint8_t, cast=True, ndim=2] mask):
    m = mask.shape[0]
    n = mask.shape[1]
    for i in range(m):
        for j in range(n):
            if mask[i, j]:
                return complex(i, j)



class Turtle(object):
    def __init__(self, start):
        self.loc = start
        self.direction = 1j
    def walk(self):
        self.loc += self.direction
    def turn_left(self):
        self.direction *= -1j
    def turn_right(self):
        self.direction *= 1j
    def get_loc(self):
        return self.loc


@cython.boundscheck(False)
@cython.wraparound(False)
def get_boundary(mask, debug=False, stop=10):
    start = get_start(mask)
    boundary_list = [start]
    boundary_set = {start}
    turtle = Turtle(start)
    if debug:
        counter = 0
    while True:
        if debug:
            print(turtle.get_loc())
        loc = turtle.get_loc()
        if mask[int(loc.real), int(loc.imag)]:
            turtle.turn_left()
        else:
            turtle.turn_right()
        turtle.walk()
        loc = turtle.get_loc()
        if mask[int(loc.real), int(loc.imag)]:
            if loc not in boundary_set:
                boundary_list.append(loc)
                boundary_set.add(loc)
        if debug:
            counter += 1
            if counter == stop:
                break
        if loc == start:
            break
    return boundary_list

In [ ]:
n = 6
r = 2
mask = circular_mask(n, r)
boundary = get_boundary(mask, debug=False)

In [ ]:
assert boundary == [(2+2j), (3+2j), (4+2j), (4+3j), (4+4j), (3+4j), (2+4j), (2+3j)]